In [ ]:
# importing necessary libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams



from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
train, test = pd.read_csv('train.csv'), pd.read_csv('test.csv')

In [ ]:
train.drop(['tweet_id'],axis = 1, inplace=True)
test.drop(['tweet_id'],axis = 1, inplace=True)

In [ ]:

pip install wordcloud

In [ ]:
from nltk.stem.porter import *
plt.style.use('seaborn')
import plotly.express as px
from plotly import graph_objs as go
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
for index, row in train.iterrows():
    tweet = row.text
    tweet = re.sub(r"https\S+", "", tweet) #remove links
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    
    train['text'].iloc[index:index+1] = tweet

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
for index, row in test.iterrows():
    tweet = row.text
    tweet = re.sub(r"https\S+", "", tweet) #remove links
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    
    test['text'].iloc[index:index+1] = tweet

In [ ]:
# Helper function to remove unwanted patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Remove Twitter handles from the data 
train['text'] = np.vectorize(remove_pattern)(train['text'], "@[\w]*")


# Remove all words below 2 characters
train['text'] = train['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# Remove punctuations, numbers, and special characters
train['text'] = train['text'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
# Helper function to remove unwanted patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Remove Twitter handles from the data 
test['text'] = np.vectorize(remove_pattern)(test['text'], "@[\w]*")


# Remove all words below 2 characters
test['text'] = test['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# Remove punctuations, numbers, and special characters
test['text'] = test['text'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
#tokenize and pad the sentences from processedText matrix
max_features = 31000 #number of words we care about
sequence_length = 256 #number of words to be taken from each sentence
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(train['text'].values)

#create train matrix
xTrain = tokenizer.texts_to_sequences(train['text'].values)
xTrain = pad_sequences(xTrain, sequence_length)

#create text matrix
xTest = tokenizer.texts_to_sequences(test['text'].values)
xTest = pad_sequences(xTest, sequence_length)
#pip install tensorflow


In [ ]:
xTrain = train['text']
xTest = test['text']

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
NUM_WORDS=9864


In [ ]:

tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
tokenizer.fit_on_texts(xTrain)
sequences_train = tokenizer.texts_to_sequences(xTrain)
sequences_Test=tokenizer.texts_to_sequences(xTest)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10794 unique tokens.


In [ ]:
xTrain = pad_sequences(sequences_train)
xTest= pad_sequences(sequences_Test, maxlen=xTrain.shape[1])

print('Shape of X train and X validation tensor:', xTrain.shape,xTest.shape)
##print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (6598, 23) (1648, 23)


In [ ]:
#pip install -U gensim


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

C:\Users\hp\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
from keras.layers import Embedding
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)

embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM)

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = xTrain.shape[1]
filterSizes = [7,8,9]
numberOfFilters = 512
dropout = 0.3

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant
import re
import tensorflow as tf
from keras.models import Model
##from ipynb.fs.full.datapreprocessing import preProcessDataset

In [ ]:
embedding_matrix.shape[0]

In [ ]:
NUM_WORDS

In [ ]:
inputs = Input(shape=(sequence_length,),dtype='int32')
embedding = Embedding(NUM_WORDS,EMBEDDING_DIM,embeddings_initializer=Constant(embedding_matrix),
                      input_length=sequence_length,trainable=True)(inputs)
reshape = Reshape((sequence_length, EMBEDDING_DIM,1))(embedding)

conv0 = Conv2D(numberOfFilters, kernel_size=(filterSizes[0], EMBEDDING_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv1 = Conv2D(numberOfFilters, kernel_size=(filterSizes[1], EMBEDDING_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv2 = Conv2D(numberOfFilters, kernel_size=(filterSizes[2], EMBEDDING_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxPool0 = MaxPool2D(pool_size=(sequence_length-filterSizes[0]+1,1), strides=(1,1), padding='valid')(conv0)
maxPool1 = MaxPool2D(pool_size=(sequence_length-filterSizes[1]+1,1), strides=(1,1), padding='valid')(conv1)
maxPool2 = MaxPool2D(pool_size=(sequence_length-filterSizes[2]+1,1), strides=(1,1), padding='valid')(conv2)

concatenatedTensor = Concatenate(axis=1)([maxPool0, maxPool1, maxPool2])
flatten = Flatten()(concatenatedTensor)
dropout = Dropout(dropout)(flatten)
output = Dense(units=1, activation='sigmoid')(dropout)

classifier = Model(inputs=inputs, outputs=output)

In [ ]:
#train the model and create predictions and store them in a dictionary

classes = ['relevance','directed_hate','generalised_hate','sarcasm','allegation','justification','refutation','Favour','oppose']
for className in classes:
    print('Training and Prediction for the class: ', className)
    classifier.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['acc'])
    classifier.fit(xTrain, train[className], batch_size=50, epochs=5, validation_split=0.2)
    

Training and Prediction for the class:  relevance
Epoch 1/5
106/106 [==============================] - 66s 619ms/step - loss: 1.1489 - acc: 0.6396 - val_loss: 0.7209 - val_acc: 0.6962
Epoch 2/5
106/106 [==============================] - 66s 624ms/step - loss: 0.4352 - acc: 0.8041 - val_loss: 0.7730 - val_acc: 0.6833
Epoch 3/5
106/106 [==============================] - 67s 628ms/step - loss: 0.3210 - acc: 0.8713 - val_loss: 0.8092 - val_acc: 0.6561
Epoch 4/5
106/106 [==============================] - 69s 653ms/step - loss: 0.2533 - acc: 0.9105 - val_loss: 0.9096 - val_acc: 0.6598
Epoch 5/5
106/106 [==============================] - 75s 707ms/step - loss: 0.2224 - acc: 0.9272 - val_loss: 0.9661 - val_acc: 0.6705
Training and Prediction for the class:  directed_hate
Epoch 1/5
106/106 [==============================] - 73s 677ms/step - loss: 0.4269 - acc: 0.9031 - val_loss: 0.1517 - val_acc: 0.9712
Epoch 2/5
106/106 [==============================] - 67s 635ms/step - loss: 0.0653 - acc: 0.